# `f3dasm`: Framework for Data-Driven Design and Analysis of Structures and Materials 
*April 3rd, 2023* <br>
*Code Release Week \#1*

# Table of contents

**1. Project introduction**
- 1.1 Overview
- 1.2 Conceptual Map and use cases
- 1.3 Computational Framework
- 1.4 Installation and getting started

**2. Demonstration**

## 1.1 Overview

f3dasm is an attempt to unite data-driven design and analysis of structures of materials.
More concretely, 

## 1.2 Conceptual map and use cases

**Using `f3dasm` to handle your design of experiments**
- *Have your own functions and modules and coat them in a `f3dasm` sauce to manage and scale-up your experiments!*

**Using `f3dasm` to benchmark or compare models**
- *Go fully `f3dasm`: use existing implementations to benchmark parts of the data-driven machine learning process!*

**Develop on `f3dasm`**
- *Work hard, play hard: work towards making your implementations an official `f3dasm`extension!*



## 1.4 Installation and getting started

### System requirements
`f3dasm` is compatible with:
1. Python 3.7 to 3.10.
2. the three major operations system (Linux, MacOS, Ubuntu).
3. the default environment of Google Colab (Python 3.8, Linux) 
4. the `pip` package manager system.

Installation instruction can be found in the documentation page under [Getting Started](https://bessagroup.github.io/F3DASM/gettingstarted.html)

You can check if the installation was succesfull by importing f3dasm. It will show the installed version and the dependencies:

In [1]:
import f3dasm

2023-03-30 10:15:43,458 - Imported f3dasm
2023-03-30 10:15:44.562616: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 10:15:44.694562: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-30 10:15:45.350305: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.1/lib64
2023-03-30 10:15:4


f3dasm:
       f3dasm: No __version__ attribute!

System:
    python: 3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:49:35)  [GCC 10.4.0]
executable: /home/martin/miniconda3/envs/f3dasm_env3/bin/python
   machine: Linux-5.14.0-1059-oem-x86_64-with-glibc2.10

Core package dependencies:
        numpy: 1.23.5
        scipy: 1.9.3
       pandas: 1.5.2
   matplotlib: 3.6.2
       pathos: 0.3.0
        hydra: 1.2.0
     autograd: No __version__ attribute!

Machine learning extension:
   tensorflow: 2.11.0

Optimization extension:
       GPyOpt: 1.2.6
          GPy: 1.10.0
   tensorflow: 2.11.0
        pygmo: 2.19.0

Sampling extension:
        SALib: 1.4.7


#### Distinction between python package and repository
- The Python  PyPI package (`pip install f3dasm`) contains the code that is used when installing the package as a **user**. It contains only the `main` branch version.
- The GitHub repository is mainly for **developers** and besides the package includes:
  - Studies (more on that later)
  - Test suite
  - Documentation source
  - Tutorial notebooks

### Extensions

- The package contains a lot of implementation for each of the blocks.
- The installation of dependencies of `f3dasm` is modular: you decide what you want to use or not.

We can distinguish three ways of using `f3dasm`:

#### Using `f3dasm` to handle your design of experiments
*Have your own functions and modules and coat them in a `f3dasm` sauce to manage and scale-up your experiments!*

The **core** package: contains the minimal installation to use `f3dasm` without extended features. 
Installed with `pip install f3dasm`

The core package contains the following features:
1. provide a way to parametrize your experiment with the **design-of-experiments** classes.
2. provide the option to investigate their experiment by **sampling** and **optimizing** their design.
3. provide the user guidance in **parallelizing** their program and ordering their data.
4. give the user ways of deploying their **experiment** at the HPC (TORQUE system)

The core package requires the following dependencies:
- `numpy` and `scipy`: for numerical operations
- `pandas`: for the representation of the design of experiments
- `matplotlib`: for plotting
- `hydra-core`: for deploying your experiment 
- `pathos`: for multiprocessing
- `autograd`: for computing gradients

#### Using `f3dasm` to benchmark or compare models
*Go fully `f3dasm`: use existing implementations to benchmark parts of the data-driven machine learning process!*

You can solely use the core package, but it is advised to enrich `f3dasm` with its **extensions** 

The extensions contain the following features:
1. provide various **implementations** to accommodate common machine learning workflows.
2. provide **adapter** classes that link common machine learning libraries to `f3dasm` base classes. 


For each of the blocks, extensions can be installed to extend the choice of implementations. Installed with `pip install f3dasm[<name of extension>]`

The following extensions are available:
- **machinelearning**: containing various `tensorflow` related models
- **sampling**: containing sampling strategies from `SALib`
- **optimization**: containing various optimizers from `GPyOpt`, `pygmo` and `tensorflow`


#### Develop on `f3dasm`
*Work hard, play hard: work towards making your implementations an official `f3dasm`extension!*

If you want your implementation to be part of the `f3dasm` package, you can develop an adapter and/or implementation for `f3dasm`

The **developement** package: contains the full installation plus requirements for developing on `f3dasm`. 
Installed with `pip install f3dasm[dev]`

Information on how to contribute to `f3dasm` can be found [on the wiki page of the GitHub repository](https://github.com/bessagroup/F3DASM/wiki)!


### Feature requirements
The first release of `f3dasm` should:
1. have a way to parametrize a **design-of-experiments** from their own program
2. give the user the option to **sample** and **optimize** from their design with various optimizers
3. give the user guidance in **parallelizing** and ordering their experiment-data
4. provide high-level classes to deploy **simulations**.
4. give the user ways of deploying their **experiment** at the HPC (TORQUE system)

The first release of `f3dasm` **should not**:
1. force the user to use the built-in machine learning models and optimizers (these are extensions)
2. include state-of-the-art machine learning models as extensions

## Requirements of first release of f3dasm

> Distinction between python package and repository
> - Python package contains the code that is used when installing the package (`pip install f3dasm`) as a user. It contains only the `main` branch version
> - The repository is mainly for developers and to showcase studies (more on that later)

### System requirements
The first release of `f3dasm` **should**:
1. comply to the Bessa Group code of conduct
2. compatible with Python 3.7 to 3.10
3. compatible with the three major operations system (Linux, MacOS, Ubuntu)
4. available as a `pip` install

The first release of `f3dasm` **should not**:
1. use `conda` packages as requirements
2. be hosted on other package distribution platforms (`poetry`, `conda`)

### Feature requirements
The first release of `f3dasm` should:
1. have a way to parametrize a **design-of-experiments** from their own program
2. give the user the option to **sample** and **optimize** from their design with various optimizers
3. give the user guidance in **parallelizing** and ordering their experiment-data
4. provide high-level classes to deploy **simulations**.
4. give the user ways of deploying their **experiment** at the HPC (TORQUE system)

The first release of `f3dasm` **should not**:
1. force the user to use the built-in machine learning models and optimizers (these are extensions)
2. include state-of-the-art machine learning models as extensions

In [2]:
import f3dasm

We can show our exact installation with the following function:

In [3]:
f3dasm.show_versions()


f3dasm:
       f3dasm: 0.2.98

System:
    python: 3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:49:35)  [GCC 10.4.0]
executable: /home/martin/miniconda3/envs/f3dasm_env3/bin/python
   machine: Linux-5.14.0-1059-oem-x86_64-with-glibc2.10

Core package dependencies:
        numpy: 1.23.5
        scipy: 1.9.3
       pandas: 1.5.2
   matplotlib: 3.6.2
       pathos: 0.3.0
        hydra: 1.2.0
     autograd: No __version__ attribute!

Machine learning extension:
   tensorflow: 2.11.0

Optimization extension:
       GPyOpt: 1.2.6
          GPy: 1.10.0
   tensorflow: 2.11.0
        pygmo: 2.19.0

Sampling extension:
        SALib: 1.4.7


### Namespace

Base classes are available from the global namespace:

In [4]:
f3dasm.Sampler, f3dasm.Optimizer, f3dasm.Model

(f3dasm.sampling.sampler.Sampler,
 f3dasm.optimization.optimizer.Optimizer,
 f3dasm.machinelearning.model.Model)

The design space features are also available from the global:

In [5]:
f3dasm.DesignSpace, f3dasm.ExperimentData, f3dasm.ContinuousParameter, f3dasm.make_nd_continuous_design

(f3dasm.design.design.DesignSpace,
 f3dasm.design.experimentdata.ExperimentData,
 f3dasm.design.parameter.ContinuousParameter,
 <function f3dasm.design.design.make_nd_continuous_design(bounds: numpy.ndarray, dimensionality: int) -> f3dasm.design.design.DesignSpace>)

Other utility functions that are under the global namespace:

In [6]:
f3dasm.run_optimization, f3dasm.show_versions

(<function f3dasm.run_optimization.run_optimization(optimizer: f3dasm.optimization.optimizer.Optimizer, function: f3dasm.functions.function.Function, sampler: f3dasm.sampling.sampler.Sampler, iterations: int, seed: int, number_of_samples: int = 30) -> f3dasm.design.experimentdata.ExperimentData>,
 <function f3dasm._show_versions.show_versions()>)

All implementations can found under the respective submodules:
- `f3dasm.sampling`
- `f3dasm.machinelearning`
- `f3dasm.optimization`
- `f3dasm.simulation`

In [7]:
f3dasm.optimization.Adam, f3dasm.sampling.RandomUniform, f3dasm.machinelearning.LinearRegression

(f3dasm.optimization.adam.Adam,
 f3dasm.sampling.randomuniform.RandomUniform,
 f3dasm.machinelearning.linear_regression.LinearRegression)

Furthermore, there two more submodules:
- `f3dasm.functions`: for benchmarkfunctions, they can be used in any stage in the process
- `f3dasm.experiment`: Experiment utilities, for datacollection, paralellization and HPC support

## Modularity

Whenever you import the f3dasm package, the implementations are only loaded when you create an instance of the class.
Only then it is checked if a particular dependency is installed or not. This keeps the package lightweight and customizable

Example, The `SALib` library is not installed. I can load the `LatinHyperCube` sampler without any problem.

In [8]:
f3dasm.sampling.LatinHypercube

f3dasm.sampling.latinhypercube.LatinHypercube

Only when I create an instance of this sampler, an ImportError is thrown:

In [9]:
sampler = f3dasm.sampling.LatinHypercube(design=0)

All the available samplers within your installation can be shown with `f3dasm.sampling.SAMPLERS`

> same logic goes for `OPTIMIZERS` and `MODELS`

In [10]:
f3dasm.sampling.SAMPLERS

[f3dasm.sampling.randomuniform.RandomUniform,
 f3dasm.sampling.latinhypercube.LatinHypercube,
 f3dasm.sampling.sobolsequence.SobolSequence]

## Other notes

- way of working with f3dasm: 'fully f3dasm' or 'core f3dasm' (base classes both protocol and parent class)
- lacking implementation of simulator
- enthousiasm from bessa group
- after release v.1.0